In [ ]:
import numpy as np
import tensorflow as tf
import os
import librosa

In [ ]:
n_mfcc = 40
max_pad_len = 173  
sample_rate = 16000
data_dir = 'audios' 
classes = ["bed", "bird", "cat", "dog", "down", "eight", "five", "four", "go", "happy", "house", "left", "marvel", "nine", "no", "off", "on", "one", "right", "seven", "sheila", "six", "stop", "three", "tree", "two", "up", "wow", "yes", "zero"]

In [ ]:
def representative_dataset_gen():
    # Obtener la lista de archivos de audio en la carpeta
    audio_files = []
    
    for class_ in classes:
        audio_files = audio_files + [os.path.join(data_dir, class_, fname) for fname in os.listdir(data_dir + '/' + class_) if fname.endswith('.wav')]
    
    print("audio_files", len(audio_files))
    
    for file_path in audio_files:
        # Cargar el archivo de audio
        x, sr = librosa.load(file_path, sr=sample_rate)
        mfccs = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=n_mfcc)
        
        # Rellenar las secuencias para que todas tengan la misma longitud
        if mfccs.shape[1] < max_pad_len:
            # Añadir ceros (padding) al final para igualar la longitud
            pad_width = max_pad_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')
        elif mfccs.shape[1] > max_pad_len:
            # Recortar la secuencia si es más larga
            mfccs = mfccs[:, :max_pad_len]
        
        mfccs = np.expand_dims(mfccs, axis=0)  # Añadir el batch size
        mfccs = np.expand_dims(mfccs, axis=-1)  # Añadir la dimensión de canales (1)
        
        yield [mfccs]

In [ ]:

new_model = tf.keras.models.load_model('keyphrase_detection_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(new_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # Tipo de inferencia de entrada
converter.inference_output_type = tf.float32
tflite_model = converter.convert()

open("model_quantized.tflite", "wb").write(tflite_model)

In [ ]:
!edgetpu_compiler model_quantized.tflite